In [1]:
import matplotlib 
matplotlib.rcParams.update({'font.size': 18})
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import gsw
from datetime import datetime,date
import os as os
from matplotlib.gridspec import GridSpec
from numpy.fft import fft,fft2,fftfreq
# importig movie py libraries
from scipy.interpolate import interp2d
from sklearn.linear_model import LinearRegression
from scipy import fftpack
from tqdm import tqdm
# analog data assimilation
from scipy.stats import linregress,norm
import dask as da
import xarray as xar
import glob as glob



# LEARNING PERIOD GRIDDED DATA

In [6]:
latmin,latmax,lonmin,lonmax=-25,25,120,180

File=xar.load_dataset('fullpacific2.nc').sel(longitude=slice(120,290))

res=3
loninires=File.coords['longitude'].diff(dim='longitude').min().values


loninimax=int(np.ceil(File.coords['longitude'].values.max()))
loninimin=int(np.floor(File.coords['longitude'].values.min()))
latinimax=int(np.ceil(File.coords['latitude'].values.max()))
latinimin=int(np.floor(File.coords['latitude'].values.min()))



bins=[x for x in np.linspace(latinimin-res*(latinimax-latinimin)%res,latinimax,(latinimax-(latinimin-res+(latinimax-latinimin)%res))//res+1)]





Filebis=xar.open_mfdataset(glob.glob('/net/alpha/exports/sciences/data/LPO_ISAS/climref_ISAS17/ISAS17_DMFD_CLIM_ann_STD_*.nc',recursive=True)[:],parallel=True,decode_times=False ).squeeze()

latmin,latmax,lonmin,lonmax=-25,25,120,180
left_part=Filebis.sel({"latitude":slice(latmin,latmax),"longitude":slice(lonmin,lonmax)})

latmin,latmax,lonmin,lonmax=-25,25,-180,-70

right_part=Filebis.sel({"latitude":slice(latmin,latmax),"longitude":slice(lonmin,lonmax)})
right_part['longitude']=right_part['longitude'].values+360





File=File.merge(xar.concat([left_part,right_part],dim='longitude').interp_like(File))

Region_coarse = File.coarsen(longitude=int(res/loninires),boundary="trim",side='left').mean().groupby_bins("latitude",bins).mean(dim='latitude').transpose("time","depth","latitude_bins","longitude")
RC = Region_coarse.rename_dims({"latitude_bins":"latitude"}).rename_vars({"latitude_bins":"latitude"}).chunk({'time':Region_coarse.coords['time'].size,'latitude':Region_coarse.coords['latitude_bins'].size,'longitude':Region_coarse.coords['longitude'].size})

JULTIME = (RC['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64')

RC = RC.assign({'time2':(JULTIME)})

RC.coords['time2']=RC.coords['time']
RC.coords['time']=JULTIME


RC['latitude']=np.asanyarray([_.mid for _ in(RC['latitude'].values)])


RC = RC.rename({'TPOT':'TEMP','TPOT_ERR':'TEMP_ERR'})




RC = RC.merge(RC[['TEMP','PSAL']].groupby("time2.month").mean().rename({'TEMP':'T_Climato','PSAL':'S_Climato'}))


res2 = 9
RC = RC.merge((RC[['TEMP','PSAL']].groupby('time2.month')-RC[['TEMP','PSAL']].groupby('time2.month').mean()).rolling({"longitude":res2//res,"latitude":res2//res},min_periods=2).mean(skipna=True).polyfit(deg=1,dim='time'))
# RC = RC.merge(((RC['TEMP'].groupby('time2.month')-RC['T_Climato']).merge(RC['PSAL'].groupby('time2.month')-RC['S_Climato'])).rolling({"longitude":res2//res,"latitude":res2//res},min_periods=2).mean(skipna=True).polyfit(deg=1,dim='time'))

# RC = RC.merge(((RC['TEMP'].groupby('time2.month')-RC['T_Climato'])-RC['TEMP_polyfit_coefficients'].sel(degree=0)-RC['time']*RC['TEMP_polyfit_coefficients'].sel(degree=1)).rename('TEMP_detrend')).merge(((RC['PSAL'].groupby('time2.month')-RC['S_Climato'])-RC['PSAL_polyfit_coefficients'].sel(degree=0)-RC['time']*RC['PSAL_polyfit_coefficients'].sel(degree=1)).rename('PSAL_detrend'))
RC = RC.merge(((RC['TEMP'].groupby('time2.month')-RC['T_Climato'])).rename('TEMP_detrend')).merge(((RC['PSAL'].groupby('time2.month')-RC['S_Climato'])).rename('PSAL_detrend'))


RC.to_netcdf('ISAS_TropicalPacific.nc','w')

/opt/linux/envs/envs/phyocean-2022.05/lib/python3.8/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing with an out-of-order index is generating 19 times more chunks
  return self.array[key]
/opt/linux/envs/envs/phyocean-2022.05/lib/python3.8/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing with an out-of-order index is generating 19 times more chunks
  return self.array[key]
/opt/linux/envs/envs/phyocean-2022.05/lib/python3.8/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing with an out-of-order index is generating 19 times more chunks
  return self.array[key]
/opt/linux/envs/envs/phyocean-2022.05/lib/python3.8/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing with an out-of-order index is generating 19 times more chunks
  return self.array[key]


# THERMODYNAMICAL COMPONENTS OVER THE LEARNING PERIOD

In [2]:
RC=xar.open_mfdataset('ISAS_TropicalPacific.nc',parallel=True,)
Alphabet = xar.open_mfdataset(glob.glob('alpha_beta/ISAS20_ARGO_*.nc'))

# Alphabet = Alphabet.sel(depth=[5,10,50,100,300,500,700,1000,1500,2000],method='nearest').sel({"latitude":slice(np.min(RC['latitude'])-1.5,np.max(RC['latitude'])+1.5),"longitude":slice(np.min(RC['longitude'])-1.5,np.max(RC['longitude'])+1.5)})
Alphabet_west = Alphabet.sel(depth=[5,10,50,100,200,300,400,500,700,1000,1500,2000],method='nearest').sel({"latitude":slice(np.min(RC['latitude'])-1.5,np.max(RC['latitude'])+1.5),"longitude":slice(np.min(RC['longitude'])-1.5,180)})
Alphabet_east = Alphabet.sel(depth=[5,10,50,100,200,300,400,500,700,1000,1500,2000],method='nearest').sel({"latitude":slice(np.min(RC['latitude'])-1.5,np.max(RC['latitude'])+1.5),"longitude":slice(-180,np.max(RC['longitude'])+1.5-360)})
Alphabet_east=Alphabet_east.assign_coords({"longitude":Alphabet_east['longitude']+360})
Alphabet = xar.concat((Alphabet_west,Alphabet_east),dim="longitude")
Alphabet =Alphabet.coarsen({'latitude':6,'longitude':6},'pad').mean().drop_isel(latitude=[17],)

Alphabet.to_netcdf('ISAS_AlphaBetaCoeffs.nc')

NameError: name 'Alphabet' is not defined

# STANDARDIZATION OF THE OBSERVATIONS

In [45]:
timemin=1930
timemax=timemin+9
ENtemp = xar.open_mfdataset('EN_OBS/ENtemp'+str(timemin)[-2:]+'Pacific.nc',parallel=True)

switch = False 
for zepth in [5,10,50,100,200,300,400,500,700,1000,1500,2000]:
    
    if zepth == 2000:
        bnd1,bnd2 = 1750,2250
        
    if zepth == 1500:
        bnd1,bnd2 = 1250,1750
        
    if zepth == 1000:
        bnd1,bnd2 = 850,1250
    elif (zepth==700):
        bnd1,bnd2 = 600,800
    elif (zepth==500):
        bnd1,bnd2 = 450,600
    elif (zepth==400):
        bnd1,bnd2 = 350,450
    elif (zepth==300):
        bnd1,bnd2 = 250,350
    elif (zepth==200):
        bnd1,bnd2 = 150,250
    elif (zepth==100):
        bnd1,bnd2 = 75,150
    elif (zepth==50):
        bnd1,bnd2 = 25,75
    elif (zepth==10):
        bnd1,bnd2 = 7,25
    else:
        bnd1,bnd2 = 0,7
        
    print(zepth)
    
    argumt=da.array.argmin(da.array.where(zepth-ENtemp['DEPH_CORRECTED'].fillna(1e5)>=0,zepth-ENtemp['DEPH_CORRECTED'].fillna(1e5),1e5),1).compute()
    
    born1=da.array.all(zepth-ENtemp['DEPH_CORRECTED'].ffill(dim='N_LEVELS').bfill(dim='N_LEVELS')<0,axis=1)
    born2=da.array.all(zepth-ENtemp['DEPH_CORRECTED'].ffill(dim='N_LEVELS').bfill(dim='N_LEVELS')>0,axis=1)

    Nprs = ENtemp.coords['N_PROF'].size
    Nzz = ENtemp.coords['N_LEVELS'].size
    
    z0,z1=ENtemp['DEPH_CORRECTED'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),argumt],ENtemp['DEPH_CORRECTED'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),np.where((1+argumt)>Nzz-1,-1+Nzz,(1+argumt))]
    # ENtemp['DEPH_CORRECTED'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),argumt],ENtemp['DEPH_CORRECTED'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),np.where((1+argumt)>399,399,(1+argumt))]
    T0,T1=ENtemp['POTM'].where(ENtemp['POTM_CORRECTED_QC']==b'1').data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),argumt],ENtemp['POTM'].where(ENtemp['POTM_CORRECTED_QC']==b'1').data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),np.where((1+argumt)>Nzz-1,Nzz-1,(1+argumt))]

    born3 = np.array([born1, born2, (z0 <bnd1), (z1>bnd2) ,(z0 > z1) ,(zepth<z0) ,(zepth>z1)]).any(axis=0)


    #     born3 = (z0-z1>=zscl)
    T0=np.where(born3,np.nan,T0)
    T1=np.where(born3,np.nan,T1)
    if switch : 
        ENewtemp = np.concatenate((ENewtemp,(np.array([T0,T1,np.empty_like(T0)])[np.nanargmin(np.abs(np.array([z0,z1,1e6*np.ones_like(T0)])-zepth),0),np.ix_(np.arange(0,Nprs,1))])),axis=0)
        # QC_z = np.concatenate((QC_z,np.where(np.isnan(T0),np.nan,(np.where(z1-z0<=zscl,0,1)+np.where(zepth-z0<zscl,0,1)+np.where(z1-zepth,0,1)))[np.newaxis,:]),axis=0)
    if not switch: 
        ENewtemp = np.array([T0,T1,np.empty_like(T0)])[np.nanargmin(np.abs(np.array([z0,z1,1e6*np.ones_like(T0)])-zepth),0),np.ix_(np.arange(0,Nprs,1))]
        # QC_z = np.where(np.isnan(T0),np.nan,(np.where(z1-z0<=zscl,0,1)+np.where(zepth-z0<zscl,0,1)+np.where(z1-zepth,0,1)))[np.newaxis,:]
        switch = True

       #
#      

5
10
50
100
200
300
400
500
700
1000
1500
2000


# FILTER OUT HIGH-FREQUENCY DATA AND CREATE SUPER-PROFILES

In [46]:
         
ENtemp2 = xar.Dataset( 
        {
            'POTM': (['depth','N_PROF'],ENewtemp),
            # 'QC' : (['depth','N_PROF'],QC_z.compute())
        },
        coords = {
            'JULD': (['N_PROF'],ENtemp['JULD'].values),
            'LATITUDE': (['N_PROF'],ENtemp['LATITUDE'].values),
            'LONGITUDE': (['N_PROF'],ENtemp['LONGITUDE'].values),
            'Depth': (['depth'],np.array([5,10,50,100,200,300,400,500,700,1000,1500,2000])),
            'PROJECT_NAME' : (['N_PROF'],ENtemp['PROJECT_NAME'].values),
            'PLATFORM_NUMBER' : (['N_PROF'], ENtemp['PLATFORM_NUMBER'].values)
        }
)


indexes = np.char.array(np.linspace(0,500,501,dtype='int').astype('str') ).zfill(3)

platform50=[ENtemp2['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENtemp2['N_PROF'].size)]
nameplatform = np.array(platform50)

condit = xar.DataArray((nameplatform == b'MRB'),dims=['N_PROF'])#((ENtemp2['PROJECT_NAME'].str[:5]==b'GTSPP')&(ENtemp2['PLATFORM_NUMBER'].dropna('N_PROF').str[2:].str.strip().astype('str').isin(indexes)))^(ENtemp2['PROJECT_NAME'].str[5:8].isin(b'MRB'))^(ENtemp2['PROJECT_NAME'].str[5:8].isin(b'UOR'))

moorObs = ENtemp2.where(condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).compute()

labofmoor = np.unique(moorObs['PLATFORM_NUMBER'])

timebins=np.arange(np.datetime64('20'+str(timemin)[-2:]+'-01-01'),np.datetime64('20'+str(timemax)[-2:]+'-12-31')+15,15)
# timebins=np.arange(np.datetime64('19'+str(timemin)[-2:]+'-01-01'),np.datetime64('19'+str(timemax)[-2:]+'-12-31')+15,15)
# timebins=np.arange(np.datetime64('1990-01-01'),np.datetime64('1999-12-31')+15,15)

if labofmoor.size!=0:
    moorObs2 = moorObs.where(moorObs['PLATFORM_NUMBER']==labofmoor[0],drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
    moorObs2 = moorObs2.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(moorObs2['JULD_bins'].values)]))})

    ii=1
    for label in labofmoor : 
        print(str(ii)+'/'+str(labofmoor.shape),end='\r')
        appendix=moorObs.where(moorObs['PLATFORM_NUMBER']==label,drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
        appendix = appendix.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(appendix['JULD_bins'].values)]))})

        moorObs2 =xar.concat((moorObs2,appendix),dim='N_PROF')
        ii+=1

    
    ENtemp3 = xar.concat((ENtemp2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['PROJECT_NAME','PLATFORM_NUMBER']),moorObs2.rename({'JULD_bins':'JULD'})),dim='N_PROF')
    
else : 
    ENtemp3 =ENtemp2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['PROJECT_NAME','PLATFORM_NUMBER'])
    
    
    
ENtemp3.to_netcdf('ENtemp_NoInterp'+str(timemin)[-2:]+'NoFMoorAdjusted.nc','w')
ENtemp2.to_netcdf('ENtemp_NoInterp'+str(timemin)[-2:]+'Adjusted.nc','w')

# IDENTIFY TYPE OF INSTRUMENT AND AFFECTS IT AN ERROR

In [6]:
def slicer(start=None, stop=None, step=1):
    return np.vectorize(lambda x: x[start:stop:step], otypes=[bytes])


In [7]:

timemin=1930
ENtemp = xar.open_mfdataset('EN_OBS/ENtemp_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform30=(slicer(5, 8)(ENtemp['PROJECT_NAME'].values))
# platform50=[ENtemp['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENtemp['N_PROF'].size)]

In [9]:

timemin=1940
ENtemp = xar.open_mfdataset('EN_OBS/ENtemp_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform40=(slicer(5, 8)(ENtemp['PROJECT_NAME'].values))
# platform50=[ENtemp['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENtemp['N_PROF'].size)]

In [10]:

timemin=1950
ENtemp = xar.open_mfdataset('EN_OBS/ENtemp_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform50=(slicer(5, 8)(ENtemp['PROJECT_NAME'].values))
# platform50=[ENtemp['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENtemp['N_PROF'].size)]

In [11]:
timemin=1960
ENtemp = xar.open_mfdataset('EN_OBS/ENtemp_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform60=(slicer(5, 8)(ENtemp['PROJECT_NAME'].values))
# platform60=[ENtemp['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENtemp['N_PROF'].size)]

In [12]:
timemin=1970
ENtemp = xar.open_mfdataset('EN_OBS/ENtemp_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform70=(slicer(5, 8)(ENtemp['PROJECT_NAME'].values))
# platform70=[ENtemp['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENtemp['N_PROF'].size)]

In [13]:
timemin=1980
ENtemp = xar.open_mfdataset('EN_OBS/ENtemp_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform80=(slicer(5, 8)(ENtemp['PROJECT_NAME'].values))
# platform80=[ENtemp['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENtemp['N_PROF'].size)]

In [14]:
timemin=1990
ENtemp = xar.open_mfdataset('EN_OBS/ENtemp_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)
# platform90=[ENtemp['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENtemp['N_PROF'].size)]
platform90=(slicer(5, 8)(ENtemp['PROJECT_NAME'].values))    # => ['el' 'ow' 're' 'ou']

In [15]:
timemin=2000
ENtemp = xar.open_mfdataset('EN_OBS/ENtemp_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)
# platform90=[ENtemp['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENtemp['N_PROF'].size)]
platform00=(slicer(5, 8)(ENtemp['PROJECT_NAME'].values))    # => ['el' 'ow' 're' 'ou']

In [123]:
np.unique(platform30)

array([b'OSD'], dtype='|S3')

In [124]:
np.unique(platform40)

array([b'MBT', b'OSD'], dtype='|S3')

In [125]:
np.unique(platform50)

array([b'MBT', b'OSD'], dtype='|S3')

In [126]:
np.unique(platform60)

array([b'CTD', b'MBT', b'OSD', b'XBT'], dtype='|S3')

In [127]:
np.unique(platform70)

array([b'CTD', b'MBT', b'MRB', b'OSD', b'XBT'], dtype='|S3')

In [128]:
np.unique(platform80)

array([b'CTD', b'MBT', b'MRB', b'OSD', b'XBT'], dtype='|S3')

In [129]:
np.unique(platform90)

array([b'BA ', b'CT ', b'CTD', b'DRB', b'MBT', b'MRB', b'OSD', b'PF ',
       b'PFL', b'TE ', b'TR ', b'UOR', b'XB ', b'XBT'], dtype='|S3')

In [130]:
np.unique(platform00)

array([b'   ', b'BA ', b'CTD', b'MBT', b'MRB', b'OSD', b'PF ', b'PFL',
       b'TE ', b'TR ', b'XB ', b'XBT'], dtype='|S3')

In [16]:
def instrErr(x):
    y = np.zeros(len(x))
    y[(x==b'CTD')^(x==b'CT ')]+= 0.002
    y[(x==b'XBT')^(x==b'XB ')]+= 0.15
    y[(x==b'MBT')^(x==b'MRB')^(x==b'BA ')]+= .3
    y[(x==b'PF ')^(x==b'PFL')]+=.005
    y[(x==b'DRB')]+=.01
    y[(x==b'UOR')]+=.01
    y[y==0]+=.02
    return y

In [297]:
test.shape

(832703,)

In [17]:
nameplatform = np.concatenate((np.array(platform30),np.array(platform40),np.array(platform50),np.array(platform60),np.array(platform70),np.array(platform80),np.array(platform90),np.array(platform00)))
test = np.concatenate((instrErr(np.array(platform30)),instrErr(np.array(platform40)),instrErr(np.array(platform50)),instrErr(np.array(platform60)),instrErr(np.array(platform70)),instrErr(np.array(platform80)),instrErr(np.array(platform90)),instrErr(np.array(platform00))))

# SAVE INTRUMENT ERRORS

In [18]:
xar.Dataset({'PLATFORM':(['N_PROF'],nameplatform),'Err':(['N_PROF'],test)}).to_netcdf('ENtemp_NoInterp_Err_moretime_Adjusted.nc')

In [19]:

ENtemp2 = xar.open_mfdataset(glob.glob('EN_OBS/ENtemp_NoInterp*0Adjusted.nc')[0])

for i in range(0,7):
    ENtemp2= xar.concat((ENtemp2,xar.open_mfdataset(glob.glob('EN_OBS/ENtemp_NoInterp*0Adjusted.nc')[1:][i])),'N_PROF')
    

# FILTER OUT HIGH-FREQUENCY DATA AND CREATE SUPER-PROFILES, THEN SAVE THE WHOLE HISTORAL SET OF TEMPERATURE OBSERVATIONS INTO ONE FILE

In [20]:
timemin=1950
timemax=1999
indexes = np.char.array(np.linspace(0,500,501,dtype='int').astype('str') ).zfill(3)


condit = xar.DataArray((nameplatform == b'MRB'),dims=['N_PROF'])#((ENtemp2['PROJECT_NAME'].str[:5]==b'GTSPP')&(ENtemp2['PLATFORM_NUMBER'].dropna('N_PROF').str[2:].str.strip().astype('str').isin(indexes)))^(ENtemp2['PROJECT_NAME'].str[5:8].isin(b'MRB'))^(ENtemp2['PROJECT_NAME'].str[5:8].isin(b'UOR'))

moorObs = ENtemp2.where(condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).compute()

labofmoor = np.unique(moorObs['PLATFORM_NUMBER'])

timebins=np.arange(np.datetime64('19'+str(timemin)[-2:]+'-01-01'),np.datetime64('19'+str(timemax)[-2:]+'-12-31')+15,15)
# timebins=np.arange(np.datetime64('1990-01-01'),np.datetime64('1999-12-31')+15,15)

if labofmoor.size!=0:
    moorObs2 = moorObs.where(moorObs['PLATFORM_NUMBER']==labofmoor[0],drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
    moorObs2 = moorObs2.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(moorObs2['JULD_bins'].values)]))})

    ii=1
    for label in labofmoor : 
        print(str(ii)+'/'+str(labofmoor.shape),end='\r')
        appendix=moorObs.where(moorObs['PLATFORM_NUMBER']==label,drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
        appendix = appendix.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(appendix['JULD_bins'].values)]))})

        moorObs2 =xar.concat((moorObs2,appendix),dim='N_PROF')
        ii+=1

    
    
    
    ENtemp3 = xar.concat((ENtemp2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['PROJECT_NAME','PLATFORM_NUMBER']),moorObs2.rename({'JULD_bins':'JULD'})),dim='N_PROF')
    
else : 
    ENtemp3 =ENtemp2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['PROJECT_NAME','PLATFORM_NUMBER'])
    
    
    
# ENtemp3.to_netcdf('ENtempNoFMoorAdjusted.nc','w')

ValueError: None of the data falls within bins with edges array(['1950-01-01', '1950-01-16', '1950-01-31', ..., '1999-12-10',
       '1999-12-25', '2000-01-09'], dtype='datetime64[D]')

In [149]:

ENtemp2.to_netcdf('ENtemp_NoInterpNoFMoorAdjusted.nc','w')

In [21]:

ENtemp2 = xar.open_mfdataset(glob.glob('ENtemp_NoInterp*0NoFMoorAdjusted.nc')[0])

for i in range(0,7):
    ENtemp2= xar.concat((ENtemp2,xar.open_mfdataset(glob.glob('ENtemp_NoInterp*0NoFMoorAdjusted.nc')[1:][i])),'N_PROF')
    

In [199]:
test2.shape

(717142,)

In [201]:
nameplatform2.shape

(717142,)

In [22]:
test2 = np.zeros((ENtemp2['N_PROF'].size))
test2[:test[~condit].size] = test[~condit]
test2[test[~condit].size:]+= .3

nameplatform2 = np.concatenate((nameplatform[~condit].copy(),np.repeat(np.array([ b'MRB']),np.abs(test2.shape[0]-nameplatform[~condit].copy().shape[0]))))
# nameplatform2=  b'MRB'


In [23]:
xar.Dataset({'PLATFORM':(['N_PROF'],nameplatform2),'Err':(['N_PROF'],test2)}).to_netcdf('ENtemp_ERR.nc')

# STANDARDIZATION OF THE OBSERVATIONS

In [52]:
timemin=1930
timemax=timemin+9
ENsal = xar.open_mfdataset('EN_OBS/ENsal'+str(timemin)[-2:]+'Pacific.nc',parallel=True)

switch = False 
for zepth in [5,10,50,100,200,300,400,500,700,1000,1500,2000]:
    
    if zepth == 2000:
        bnd1,bnd2 = 1750,2250
        
    if zepth == 1500:
        bnd1,bnd2 = 1250,1750
        
    if zepth == 1000:
        bnd1,bnd2 = 850,1250
    elif (zepth==700):
        bnd1,bnd2 = 600,800
    elif (zepth==500):
        bnd1,bnd2 = 450,600
    elif (zepth==400):
        bnd1,bnd2 = 350,450
    elif (zepth==300):
        bnd1,bnd2 = 250,350
    elif (zepth==200):
        bnd1,bnd2 = 150,250
    elif (zepth==100):
        bnd1,bnd2 = 75,150
    elif (zepth==50):
        bnd1,bnd2 = 25,75
    elif (zepth==10):
        bnd1,bnd2 = 7,25
    else:
        bnd1,bnd2 = 0,7
        
    print(zepth)
    
    argumt=da.array.argmin(da.array.where(zepth-ENsal['DEPH_CORRECTED'].fillna(1e5)>=0,zepth-ENsal['DEPH_CORRECTED'].fillna(1e5),1e5),1).compute()
    
    born1=da.array.all(zepth-ENsal['DEPH_CORRECTED'].ffill(dim='N_LEVELS').bfill(dim='N_LEVELS')<0,axis=1)
    born2=da.array.all(zepth-ENsal['DEPH_CORRECTED'].ffill(dim='N_LEVELS').bfill(dim='N_LEVELS')>0,axis=1)

    Nprs = ENsal.coords['N_PROF'].size
    Nzz = ENsal.coords['N_LEVELS'].size
    
    z0,z1=ENsal['DEPH_CORRECTED'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),argumt],ENsal['DEPH_CORRECTED'].data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),np.where((1+argumt)>Nzz-1,Nzz-1,(1+argumt))]
    T0,T1=ENsal['PSAL'].where(ENsal['PSAL_CORRECTED_QC']==b'1').data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),argumt],ENsal['PSAL'].where(ENsal['PSAL_CORRECTED_QC']==b'1').data.vindex[np.linspace(0,Nprs-1,Nprs,dtype='int'),np.where((1+argumt)>Nzz-1,Nzz-1,(1+argumt))]

    born3 = np.array([born1, born2, (z0 <bnd1), (z1>bnd2) ,(z0 > z1) ,(zepth<z0) ,(zepth>z1)]).any(axis=0)


    #     born3 = (z0-z1>=zscl)
    T0=np.where(born3,np.nan,T0)
    T1=np.where(born3,np.nan,T1)
    if switch : 
        ENewsal = np.concatenate((ENewsal,(np.array([T0,T1,np.empty_like(T0)])[np.nanargmin(np.abs(np.array([z0,z1,1e6*np.ones_like(T0)])-zepth),0),np.ix_(np.arange(0,Nprs,1))])),axis=0)
        # QC_z = np.concatenate((QC_z,np.where(np.isnan(T0),np.nan,(np.where(z1-z0<=zscl,0,1)+np.where(zepth-z0<zscl,0,1)+np.where(z1-zepth,0,1)))[np.newaxis,:]),axis=0)
    if not switch: 
        ENewsal = np.array([T0,T1,np.empty_like(T0)])[np.nanargmin(np.abs(np.array([z0,z1,1e6*np.ones_like(T0)])-zepth),0),np.ix_(np.arange(0,Nprs,1))]
        # QC_z = np.where(np.isnan(T0),np.nan,(np.where(z1-z0<=zscl,0,1)+np.where(zepth-z0<zscl,0,1)+np.where(z1-zepth,0,1)))[np.newaxis,:]
        switch = True

       #
#      

5
10
50
100
200
300
400
500
700
1000
1500
2000


# FILTER OUT HIGH-FREQUENCY DATA AND CREATE SUPER-PROFILES

In [203]:
         
ENsal2 = xar.Dataset( 
        {
            'PSAL': (['depth','N_PROF'],ENewsal),
            # 'QC' : (['depth','N_PROF'],QC_z.compute())
        },
        coords = {
            'JULD': (['N_PROF'],ENsal['JULD'].values),
            'LATITUDE': (['N_PROF'],ENsal['LATITUDE'].values),
            'LONGITUDE': (['N_PROF'],ENsal['LONGITUDE'].values),
            'Depth': (['depth'],np.array([5,10,50,100,200,300,400,500,700,1000,1500,2000])),
            'PROJECT_NAME' : (['N_PROF'],ENsal['PROJECT_NAME'].values),
            'PLATFORM_NUMBER' : (['N_PROF'], ENsal['PLATFORM_NUMBER'].values)
        }
)


indexes = np.char.array(np.linspace(0,500,501,dtype='int').astype('str') ).zfill(3)

platform50=[ENsal2['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENsal2['N_PROF'].size)]
nameplatform = np.array(platform50)

condit = xar.DataArray((nameplatform == b'MRB'),dims=['N_PROF'])#((ENsal2['PROJECT_NAME'].str[:5]==b'GTSPP')&(ENsal2['PLATFORM_NUMBER'].dropna('N_PROF').str[2:].str.strip().astype('str').isin(indexes)))^(ENsal2['PROJECT_NAME'].str[5:8].isin(b'MRB'))^(ENsal2['PROJECT_NAME'].str[5:8].isin(b'UOR'))

moorObs = ENsal2.where(condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).compute()

labofmoor = np.unique(moorObs['PLATFORM_NUMBER'])

# timebins=np.arange(np.datetime64('20'+str(timemin)[-2:]+'-01-01'),np.datetime64('20'+str(timemax)[-2:]+'-12-31')+15,15)
timebins=np.arange(np.datetime64('19'+str(timemin)[-2:]+'-01-01'),np.datetime64('19'+str(timemax)[-2:]+'-12-31')+15,15)
# timebins=np.arange(np.datetime64('1990-01-01'),np.datetime64('1999-12-31')+15,15)

if labofmoor.size!=0:
    moorObs2 = moorObs.where(moorObs['PLATFORM_NUMBER']==labofmoor[0],drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
    moorObs2 = moorObs2.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(moorObs2['JULD_bins'].values)]))})

    ii=1
    for label in labofmoor : 
        print(str(ii)+'/'+str(labofmoor.shape),end='\r')
        appendix=moorObs.where(moorObs['PLATFORM_NUMBER']==label,drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
        appendix = appendix.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(appendix['JULD_bins'].values)]))})

        moorObs2 =xar.concat((moorObs2,appendix),dim='N_PROF')
        ii+=1

    
    
    
    ENsal3 = xar.concat((ENsal2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['PROJECT_NAME','PLATFORM_NUMBER']),moorObs2.rename({'JULD_bins':'JULD'})),dim='N_PROF')
    
else : 
    ENsal3 =ENsal2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['PROJECT_NAME','PLATFORM_NUMBER'])
    
    
    
ENsal3.to_netcdf('ENsal_NoInterp'+str(timemin)[-2:]+'NoFMoorAdjusted.nc','w')
ENsal2.to_netcdf('ENsal_NoInterp'+str(timemin)[-2:]+'Adjusted.nc','w')

ValueError: conflicting sizes for dimension 'N_PROF': length 80281 on 'JULD' and length 3610 on {'depth': 'PSAL', 'N_PROF': 'PSAL'}

# IDENTIFY TYPE OF INSTRUMENT AND AFFECTS IT AN ERROR

In [204]:
def slicer(start=None, stop=None, step=1):
    return np.vectorize(lambda x: x[start:stop:step], otypes=[bytes])


In [205]:

timemin=1930
ENsal = xar.open_mfdataset('ENsal_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform30=(slicer(5, 8)(ENsal['PROJECT_NAME'].values))
# platform50=[ENsal['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENsal['N_PROF'].size)]

In [206]:

timemin=1940
ENsal = xar.open_mfdataset('ENsal_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform40=(slicer(5, 8)(ENsal['PROJECT_NAME'].values))
# platform50=[ENsal['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENsal['N_PROF'].size)]

In [207]:

timemin=1950
ENsal = xar.open_mfdataset('ENsal_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform50=(slicer(5, 8)(ENsal['PROJECT_NAME'].values))
# platform50=[ENsal['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENsal['N_PROF'].size)]

In [208]:
timemin=1960
ENsal = xar.open_mfdataset('ENsal_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform60=(slicer(5, 8)(ENsal['PROJECT_NAME'].values))
# platform60=[ENsal['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENsal['N_PROF'].size)]

In [209]:
timemin=1970
ENsal = xar.open_mfdataset('ENsal_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform70=(slicer(5, 8)(ENsal['PROJECT_NAME'].values))
# platform70=[ENsal['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENsal['N_PROF'].size)]

In [210]:
timemin=1980
ENsal = xar.open_mfdataset('ENsal_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)

platform80=(slicer(5, 8)(ENsal['PROJECT_NAME'].values))
# platform80=[ENsal['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENsal['N_PROF'].size)]

In [211]:
timemin=1990
ENsal = xar.open_mfdataset('ENsal_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)
# platform90=[ENsal['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENsal['N_PROF'].size)]
platform90=(slicer(5, 8)(ENsal['PROJECT_NAME'].values))    # => ['el' 'ow' 're' 'ou']

In [212]:
timemin=2000
ENsal = xar.open_mfdataset('ENsal_NoInterp'+str(timemin)[-2:]+'Adjusted.nc',parallel=True)
# platform90=[ENsal['PROJECT_NAME'].values[ind][5:8] for ind in range(0,ENsal['N_PROF'].size)]
platform00=(slicer(5, 8)(ENsal['PROJECT_NAME'].values))    # => ['el' 'ow' 're' 'ou']

In [179]:
np.unique(platform30)

array([b'OSD'], dtype='|S3')

In [85]:
np.unique(platform40)

array([b'MBT', b'OSD'], dtype='|S3')

In [86]:
np.unique(platform50)

array([b'MBT', b'OSD'], dtype='|S3')

In [87]:
np.unique(platform60)

array([b'CTD', b'MBT', b'OSD', b'XBT'], dtype='|S3')

In [88]:
np.unique(platform70)

array([b'CTD', b'MBT', b'MRB', b'OSD', b'XBT'], dtype='|S3')

In [89]:
np.unique(platform80)

array([b'CTD', b'MBT', b'MRB', b'OSD', b'XBT'], dtype='|S3')

In [90]:
np.unique(platform90)

array([b'BA ', b'CT ', b'CTD', b'DRB', b'MBT', b'MRB', b'OSD', b'PF ',
       b'PFL', b'TE ', b'TR ', b'UOR', b'XB ', b'XBT'], dtype='|S3')

In [68]:
np.unique(platform00)

array([b'   ', b'BA ', b'CTD', b'MBT', b'MRB', b'OSD', b'PF ', b'PFL',
       b'TE ', b'TR ', b'XB ', b'XBT'], dtype='|S3')

CTD pre-1980 (Conductivity, Temperature, Depth) 0.02
CTD post-1980 calibrated  0.002
Profiling floats (Argo***)0.01
XCTDpre-1998 (Expendable Conductivity,Temperature, Depth) 0.08
XCTD post-1998  0.03

Christopher P. Atkinson et al 2014

EN.4.0.1 	CTD 	- 	- 	- 	- 	0.005 	
EN.4.0.1 	STD 	- 	- 	- 	- 	0.005 	
EN.4.0.1 	XCTD 	- 	- 	- 	- 	0.05 	
EN.4.0.1 	Bottle/rosette/net 	- 	- 	- 	- 	0.05 
EN.4.0.1 	Profiling float 	- 	- 	- 	- 	0.005 
EN.4.0.1 	Moored buoy 	- 	- 	- 	- 	0.02 
EN.4.0.1 	Profiling drifting buoy 	- 	- 	- 	- 	0.01 
EN.4.0.1 	Towed CTD (UOR) 	- 	- 	- 	- 	0.01 
EN.4.0.1 	Animal-mounted 	- 	- 	- 	- 	0.005 
EN.4.0.1 	Glider 	- 	- 	- 	- 	0.005



UOR = Undulating Oceanographic recorder
OSD = Ocean Station Data
TE = Tesac Message (Station Data = OSD)

In [ ]:
np.unique(platform90[~np.isnan(ENsal['POTM'].values).all(0)],return_counts=True)

(array([b'CT ', b'CTD', b'MRB', b'OSD', b'PF ', b'TE ', b'UOR'],
       dtype='|S3'),
 array([    9, 15160,  7755,  6164,    31,   690,  2523]))

In [178]:
np.unique(platform90[np.isnan(ENsal['POTM'].values).all(0)],return_counts=True)

(array([b'BA ', b'CT ', b'CTD', b'DRB', b'MBT', b'MRB', b'OSD', b'PF ',
        b'PFL', b'TE ', b'TR ', b'UOR', b'XB ', b'XBT'], dtype='|S3'),
 array([  2436,      2,    282,     16,   9090, 171935,   2429,     82,
          1104,     38,   4737,     61,   9696, 113476]))

In [91]:
def instrErr(x):
    y = np.zeros(len(x))
    y[(x==b'CTD')^(x==b'CT ')]+= 0.005
    y[(x==b'XBT')^(x==b'XB ')^(x==b'MBT')^(x==b'BA ')]+= 0
    y[(x==b'MRB')]+= .02
    y[(x==b'PF ')^(x==b'PFL')]+=0.005
    y[(x==b'DRB')]+=0.01
    y[(x==b'UOR')^(x==b'TE ')]+=.01
    y[y==0]+=.0
    return y

In [93]:
instrErr(np.unique(platform60))

array([0.005, 0.   , 0.   , 0.   ])

In [213]:
nameplatform = np.concatenate((np.array(platform30),np.array(platform40),np.array(platform50),np.array(platform60),np.array(platform70),np.array(platform80),np.array(platform90),np.array(platform00)))
test = np.concatenate((instrErr(np.array(platform30)),instrErr(np.array(platform40)),instrErr(np.array(platform50)),instrErr(np.array(platform60)),instrErr(np.array(platform70)),instrErr(np.array(platform80)),instrErr(np.array(platform90)),instrErr(np.array(platform00))))

# SAVE INTRUMENT ERRORS

In [71]:
xar.Dataset({'PLATFORM':(['N_PROF'],nameplatform),'Err':(['N_PROF'],test)}).to_netcdf('ENsal_ERR.nc')

# FILTER OUT HIGH-FREQUENCY DATA AND CREATE SUPER-PROFILES, THEN SAVE THE WHOLE HISTORAL SET OF OBSERVATIONS INTO ONE FILE

In [214]:

ENsal2 = xar.open_mfdataset(glob.glob('ENsal_NoInterp*0Adjusted.nc')[0]).rename({"POTM":"PSAL"})

for i in range(0,7):
    if i<=3:
        ENsal2= xar.concat((ENsal2,xar.open_mfdataset(glob.glob('ENsal_NoInterp*0Adjusted.nc')[1:][i]).rename({"POTM":"PSAL"})),'N_PROF')
    else : 
        ENsal2= xar.concat((ENsal2,xar.open_mfdataset(glob.glob('ENsal_NoInterp*0Adjusted.nc')[1:][i])),'N_PROF')
timemin=1930
timemax=2002
indexes = np.char.array(np.linspace(0,500,501,dtype='int').astype('str') ).zfill(3)


condit = xar.DataArray((nameplatform == b'MRB'),dims=['N_PROF'])#((ENsal2['PROJECT_NAME'].str[:5]==b'GTSPP')&(ENsal2['PLATFORM_NUMBER'].dropna('N_PROF').str[2:].str.strip().astype('str').isin(indexes)))^(ENsal2['PROJECT_NAME'].str[5:8].isin(b'MRB'))^(ENsal2['PROJECT_NAME'].str[5:8].isin(b'UOR'))

moorObs = ENsal2.where(condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).compute()

labofmoor = np.unique(moorObs['PLATFORM_NUMBER'])

timebins=np.arange(np.datetime64('19'+str(timemin)[-2:]+'-01-01'),np.datetime64('20'+str(timemax)[-2:]+'-12-31')+15,15)
# timebins=np.arange(np.datetime64('19'+str(timemin)[-2:]+'-01-01'),np.datetime64('19'+str(timemax)[-2:]+'-12-31')+15,15)
# timebins=np.arange(np.datetime64('1990-01-01'),np.datetime64('1999-12-31')+15,15)

if labofmoor.size!=0:
    moorObs2 = moorObs.where(moorObs['PLATFORM_NUMBER']==labofmoor[0],drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
    moorObs2 = moorObs2.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(moorObs2['JULD_bins'].values)]))})

    ii=1
    for label in labofmoor : 
        print(str(ii)+'/'+str(labofmoor.shape),end='\r')
        appendix=moorObs.where(moorObs['PLATFORM_NUMBER']==label,drop=True).groupby_bins(group='JULD',bins=timebins).mean(skipna=True).dropna('JULD_bins',how='all').rename_dims({'JULD_bins':'N_PROF'})
        appendix = appendix.assign_coords({'JULD_bins':(['N_PROF'],np.asanyarray([_.mid for _ in(appendix['JULD_bins'].values)]))})

        moorObs2 =xar.concat((moorObs2,appendix),dim='N_PROF')
        ii+=1
    
    
    ENsal3 = xar.concat((ENsal2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['PROJECT_NAME','PLATFORM_NUMBER']),moorObs2.rename({'JULD_bins':'JULD'})),dim='N_PROF')
    
else : 
    ENsal3 =ENsal2.where(~condit,drop=True).reset_coords(['LATITUDE','LONGITUDE']).drop(['PROJECT_NAME','PLATFORM_NUMBER'])
    
    

KeyboardInterrupt: 

In [152]:

ENsal2.to_netcdf('ENsal_NoInterpNoFMoorAdjusted.nc','w')

In [218]:

ENsal2 = xar.open_mfdataset(glob.glob('ENsal_NoInterp*0NoFMoorAdjusted.nc')[0]).rename({"POTM":"PSAL"})

for i in range(0,7):

    if i<=3:
        ENsal2= xar.concat((ENsal2,xar.open_mfdataset(glob.glob('ENsal_NoInterp*0NoFMoorAdjusted.nc')[1:][i]).rename({"POTM":"PSAL"})),'N_PROF')
    else : 
        ENsal2= xar.concat((ENsal2,xar.open_mfdataset(glob.glob('ENsal_NoInterp*0NoFMoorAdjusted.nc')[1:][i])),'N_PROF')
    

In [219]:
test2 = np.zeros((ENsal2['N_PROF'].size))
test2[:test[~condit].size] = test[~condit]
test2[test[~condit].size:]+= .3

nameplatform2 = np.concatenate((nameplatform[~condit].copy(),np.repeat(np.array([ b'MRB']),np.abs(test2.shape[0]-nameplatform[~condit].shape[0]))))
# nameplatform2=  b'MRB'


In [220]:
xar.Dataset({'PLATFORM':(['N_PROF'],nameplatform2),'Err':(['N_PROF'],test2)}).to_netcdf('ENsal_NoInterp_Err_moretime_NoFMoorAdjusted.nc')

In [143]:
ENsal3.to_netcdf('ENsal_NoInterpNoFMoorAdjusted.nc','w')

# EN4 DATA TREATMENT

In [11]:
test=xar.open_mfdataset(glob.glob("/EN4/ANALYSIS/*/*.nc"),parallel=True).interp(depth=RC['depth'],method="linear", kwargs={"fill_value": "extrapolate"})

res=1

latmin,latmax,lonmin,lonmax=np.floor(RC['latitude'].min()).values-res,np.ceil(RC['latitude'].max()).values+res,np.floor(RC['longitude'].min()).values-res,np.ceil(RC['longitude'].max()).values+res

    
bins=[x for x in np.linspace(latmin-res*(latmax-latmin)%res,latmax,int((-latmin+(latmax+res*(latmax-latmin)%res)))//res+1)]

EN4test=test['temperature'].sel({'lat':slice(latmin,latmax),'lon':slice(lonmin,lonmax)}).coarsen({'lon':res},"trim").mean().groupby_bins('lat',bins).mean().rename({'lat_bins':'lat'})
EN4test['lat']=np.asanyarray([_.mid for _ in(EN4test['lat'].values)])
EN4teststd=test['temperature_uncertainty'].sel({'lat':slice(latmin,latmax),'lon':slice(lonmin,lonmax)}).coarsen({'lon':res},"trim").mean().groupby_bins('lat',bins).mean().rename({'lat_bins':'lat'})
EN4teststd['lat']=np.asanyarray([_.mid for _ in(EN4teststd['lat'].values)])
EN4test=EN4test-273.15 #-ds3['temp_bck'].values

EN4test=EN4test.interp({'lat':np.linspace(RC['latitude'].values[0],RC['latitude'].values[-1],37),'lon':np.linspace(RC['longitude'][0],RC['longitude'][-1],308-101+1)}).compute()
EN4teststd=EN4teststd.interp({'lat':np.linspace(RC['latitude'].values[0],RC['latitude'].values[-1],37),'lon':np.linspace(RC['longitude'][0],RC['longitude'][-1],308-101+1)}).compute()

# EN4test=EN4test.interp_like(RC['TEMP'][0,0].squeeze().rename({'latitude':'lat','longitude':'lon'})).compute()
# EN4teststd=EN4teststd.interp_like(RC['TEMP'][0,0].squeeze().rename({'latitude':'lat','longitude':'lon'})).compute()

# EN4test2=test['salinity'].sel({'lat':slice(latmin,latmax),'lon':slice(lonmin+360,lonmax+360)})[:,0].coarsen({'lon':2},"trim").mean().coarsen({'lat':2},"trim").mean()
# EN4teststd2=test['salinity_uncertainty'].sel({'lat':slice(latmin,latmax),'lon':slice(lonmin+360,lonmax+360)})[:,0].coarsen({'lon':2},"trim").mean().coarsen({'lat':2},"trim").mean()
# EN4test2=EN4test2-ds3['psal_bck'].values



In [12]:
EN4clim = EN4test.groupby("time.month").mean().rename({'EN4_Climato'})

EN4test = xar.Dataset({'Temp':EN4test.groupby('time.month')-EN4clim,'Climato':EN4clim,'Std':EN4teststd}).assign({'time2':((EN4test['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64'))})

JULTIME=((EN4test['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64'))
EN4test.coords['time2']=EN4test.coords['time']
EN4test.coords['time']=JULTIME


EN4test.to_netcdf('EN4_FULL_Pac.nc','w')

In [8]:
RC=xar.open_mfdataset('RC_Pac_TPOT_moredepths.nc')

test=xar.open_mfdataset(glob.glob("/EN4/ANALYSIS/*/*.nc"),parallel=True)
test=test.interp(depth=RC['depth'],method="linear", kwargs={"fill_value": "extrapolate"})

res=1

latmin,latmax,lonmin,lonmax=np.floor(RC['latitude'].min()).values-res,np.ceil(RC['latitude'].max()).values+res,np.floor(RC['longitude'].min()).values-res,np.ceil(RC['longitude'].max()).values+res

    
bins=[x for x in np.linspace(latmin-res*(latmax-latmin)%res,latmax,int((-latmin+(latmax+res*(latmax-latmin)%res)))//res+1)]

EN4test=test['salinity'].sel({'lat':slice(latmin,latmax),'lon':slice(lonmin,lonmax)}).coarsen({'lon':res},"trim").mean().groupby_bins('lat',bins).mean().rename({'lat_bins':'lat'})
EN4test['lat']=np.asanyarray([_.mid for _ in(EN4test['lat'].values)])
EN4teststd=test['salinity_uncertainty'].sel({'lat':slice(latmin,latmax),'lon':slice(lonmin,lonmax)}).coarsen({'lon':res},"trim").mean().groupby_bins('lat',bins).mean().rename({'lat_bins':'lat'})
EN4teststd['lat']=np.asanyarray([_.mid for _ in(EN4teststd['lat'].values)])


EN4test=EN4test.interp({'lat':np.linspace(RC['latitude'].values[0],RC['latitude'].values[-1],37),'lon':np.linspace(RC['longitude'][0],RC['longitude'][-1],308-101+1)}).compute()
EN4teststd=EN4teststd.interp({'lat':np.linspace(RC['latitude'].values[0],RC['latitude'].values[-1],37),'lon':np.linspace(RC['longitude'][0],RC['longitude'][-1],308-101+1)}).compute()

# EN4test=EN4test.interp_like(RC['TEMP'][0,0].squeeze().rename({'latitude':'lat','longitude':'lon'})).compute()
# EN4teststd=EN4teststd.interp_like(RC['TEMP'][0,0].squeeze().rename({'latitude':'lat','longitude':'lon'})).compute()

# EN4test2=test['salinity'].sel({'lat':slice(latmin,latmax),'lon':slice(lonmin+360,lonmax+360)})[:,0].coarsen({'lon':2},"trim").mean().coarsen({'lat':2},"trim").mean()
# EN4teststd2=test['salinity_uncertainty'].sel({'lat':slice(latmin,latmax),'lon':slice(lonmin+360,lonmax+360)})[:,0].coarsen({'lon':2},"trim").mean().coarsen({'lat':2},"trim").mean()
# EN4test2=EN4test2-ds3['psal_bck'].values



In [9]:
EN4clim = EN4test.groupby("time.month").mean().rename({'EN4_SClimato'})

EN4test = xar.Dataset({'Psal':EN4test.groupby('time.month')-EN4clim,'SClimato':EN4clim,'SStd':EN4teststd}).assign({'time2':((EN4test['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64'))})

JULTIME=((EN4test['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64'))
EN4test.coords['time2']=EN4test.coords['time']
EN4test.coords['time']=JULTIME


EN4test.to_netcdf('EN4_Salinity_FULL_Pac.nc','w')

# IAP DATA TREATMENT

In [28]:
RC=xar.open_mfdataset('RC_Pac_TPOT_moredepths.nc')


import glob as glob 


IAP = xar.open_mfdataset(glob.glob("IAP/*/*Temp*.nc"))

res=1

latmin,latmax,lonmin,lonmax=np.floor(RC['latitude'].min()).values-res,np.ceil(RC['latitude'].max()).values+res,np.floor(RC['longitude'].min()).values-res,np.ceil(RC['longitude'].max()).values+res


# bins=[x for x in np.linspace(latmin-res*(latmax-latmin)%res,latmax,int((-latmin+(latmax+res*(latmax-latmin)%res)))//res+1)]

IAPtest=IAP.where((IAP['lat']>=latmin)&(IAP['lat']<=latmax)&(IAP['lon']>=lonmin)&(IAP['lon']<=lonmax),drop=True)
# CORAtest=CORAtest.assign_coords({'x':CORAtest['longitude'].isel(y=0),'y':CORAtest['latitude'].isel(x=0)}).rename({'x':'lon','y':'lat'})

IAPtest = IAPtest.assign({'time':(['time'],np.arange(np.datetime64('1940-01'),np.datetime64('2023-01'),np.timedelta64('1','M')))})

IAPtest=IAPtest.sel({'depth_std':slice(0,2010)}).interp({'lat':np.arange(RC['latitude'].values[0],RC['latitude'].values[-1]+1,1),'lon':np.arange(RC['longitude'].values[0],RC['longitude'].values[-1]+1,1)}).compute()

# IAPtest.interp({'lat':np.arange(RC['latitude'].values[0],RC['latitude'].values[-1]+1,1),'lon':np.arange(RC['longitude'].values[0],RC['longitude'].values[-1]+1,1)}).compute()


IAPclim = IAPtest['temp'].groupby("time.month").mean().rename({'Climato'})

IAPtest = xar.Dataset({'Psal':IAPtest['temp'].groupby('time.month')-IAPclim,'Climato':IAPclim}).assign({'time2':((IAPtest['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64'))})

JULTIME=((IAPtest['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64'))
IAPtest.coords['time2']=IAPtest.coords['time']


IAPtest.to_netcdf('IAP_Temperature_1degree_Pac.nc','w')

/tmp/ipykernel_1813272/734563555.py:18: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  IAPtest.assign({'time':(['time'],np.arange(np.datetime64('1940-01'),np.datetime64('2023-01'),np.timedelta64('1','M')))})
/tmp/ipykernel_1813272/734563555.py:18: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArr

<xarray.Dataset>
Dimensions:    (time: 996, lat: 50, lon: 168, depth_std: 119)
Coordinates:
  * lat        (lat) float32 -24.5 -23.5 -22.5 -21.5 ... 21.5 22.5 23.5 24.5
  * lon        (lon) float32 120.5 121.5 122.5 123.5 ... 284.5 285.5 286.5 287.5
  * time       (time) datetime64[ns] 1940-01-01 1940-02-01 ... 2022-12-01
  * depth_std  (depth_std) float32 1.0 5.0 10.0 15.0 ... 5.8e+03 5.9e+03 6e+03
Data variables:
    temp       (time, lat, lon, depth_std) float32 dask.array<chunksize=(1, 50, 168, 119), meta=np.ndarray>
Attributes:
    Title:           IAP 3-Dimentional Subsurface Temperature Dataset Using O...
    StartYear:       1940
    StartMonth:      1
    StartDay:        1
    EndYear:         1940
    EndMonth:        1
    EndDay:          30
    Period:          1
    GridProjection:  Mercator, gridded
    GridPoints:      360x180
    Creator:         Lijing Cheng From IAP,CAS,P.R.China
    Reference:       Cheng and Zhu 2016. Journal of Climate; Cheng et al. 201...

In [49]:
RC=xar.open_mfdataset('RC_Pac_TPOT_moredepths.nc')


import glob as glob 


IAP = xar.open_mfdataset(glob.glob("IAP/*/*salinit*.nc"))

res=1

latmin,latmax,lonmin,lonmax=np.floor(RC['latitude'].min()).values-res,np.ceil(RC['latitude'].max()).values+res,np.floor(RC['longitude'].min()).values-res,np.ceil(RC['longitude'].max()).values+res


# bins=[x for x in np.linspace(latmin-res*(latmax-latmin)%res,latmax,int((-latmin+(latmax+res*(latmax-latmin)%res)))//res+1)]

IAPtest=IAP.where((IAP['lat']>=latmin)&(IAP['lat']<=latmax)&(IAP['lon']>=lonmin)&(IAP['lon']<=lonmax),drop=True)
# CORAtest=CORAtest.assign_coords({'x':CORAtest['longitude'].isel(y=0),'y':CORAtest['latitude'].isel(x=0)}).rename({'x':'lon','y':'lat'})

IAPtest = IAPtest.assign({'time':(['time'],np.arange(np.datetime64('1940-01'),np.datetime64('2022-01'),np.timedelta64('1','M')))})

IAPtest=IAPtest.sel({'depth_std':slice(0,2010)}).interp({'lat':np.arange(RC['latitude'].values[0],RC['latitude'].values[-1]+1,1),'lon':np.arange(RC['longitude'].values[0],RC['longitude'].values[-1]+1,1)}).compute()

# IAPtest.interp({'lat':np.arange(RC['latitude'].values[0],RC['latitude'].values[-1]+1,1),'lon':np.arange(RC['longitude'].values[0],RC['longitude'].values[-1]+1,1)}).compute()


IAPclim = IAPtest['salinity'].groupby("time.month").mean().rename({'SClimato'})

IAPtest = xar.Dataset({'Psal':IAPtest['salinity'].groupby('time.month')-IAPclim,'SClimato':IAPclim}).assign({'time2':((IAPtest['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64'))})

JULTIME=((IAPtest['time'].data.astype('datetime64[D]')-np.datetime64('1950-01-01')).astype('timedelta64[D]').astype('float64'))
IAPtest.coords['time2']=IAPtest.coords['time']


IAPtest.to_netcdf('IAP_Salinity_1degree_Pac.nc','w')

/tmp/ipykernel_1813272/3221523542.py:19: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  IAPtest = IAPtest.assign({'time':(['time'],np.arange(np.datetime64('1940-01'),np.datetime64('2022-01'),np.timedelta64('1','M')))})
/tmp/ipykernel_1813272/3221523542.py:19: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to

KeyError: 'psal'